Code pour projet crypto

Import des librairies

In [22]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets


## Exercice 1

### Partie 1

In [23]:

def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()
        
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=5):
    

    # Première partie : Trouver la chaîne
    chaine_trouvee, compteur_trouve, hash_trouve = trouver_chaine(base, zeros_cibles)

    # Deuxième partie : Statistiques de temps
    temps_n = []
    temps_n_plus_1 = []
    
    for _ in range(iterations_stat):
        debut = time.time()
        trouver_chaine(base, zeros_cibles)
        fin = time.time()
        temps_n.append(fin - debut)

        debut = time.time()
        trouver_chaine(base, zeros_cibles + 1)
        fin = time.time()
        temps_n_plus_1.append(fin - debut)

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "chaine_trouvee": chaine_trouvee,
        "hash_trouve": hash_trouve,
        "compteur_trouve": compteur_trouve,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n_plus_1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


{'chaine_trouvee': 'Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA677031',
 'hash_trouve': '31e6f8e6377ad1c9895ef7c22b0ef9a252c0f0ec9d062178c58b416bcb470000',
 'compteur_trouve': 77031,
 'temps_moyen_n': 0.1520848751068115,
 'temps_moyen_n_plus_1': 0.49832873344421386,
 'rapport_temps': 3.2766488652749333}

## Exercice 2

In [24]:
with open("format.txt", "r",encoding="utf-8") as file:
    text = str(file.read())
text

"Le marché des sneakers en France a connu une croissance exponentielle au cours des dernières années, devenant un phénomène de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumière ses dynamiques économiques, culturelles et sociales.\nD'un point de vue économique, le commerce de sneakers en France est stimulé par une demande croissante pour des modèles rares et exclusifs. \nLes éditions limitées et les collaborations entre marques de renom et designers célèbres créent une urgence d'achat, poussant les prix sur le marché secondaire à atteindre des sommets vertigineux. \nCette situation a engendré l'émergence d'une véritable économie parallèle, où les sneakers sont non seulement des objets de mode, mais également des actifs spéculatifs.\n\nSur le plan culturel, les sneakers occupent une place prépondérante dans l'univers de la mode et de la 

In [25]:
password = "123456789"
passwordSalt = os.urandom(16)
key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
iv = secrets.randbits(256)
#On est obliger de recrée un mot de pass unique pour chiffrer le texte
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(text)

In [26]:
print(aes.decrypt(ciphertext))

b'\xd6c\xc5\xf0\xba\xd6\x1baA\xa52\xbf9\xb8\xea\xd5\xb3\x96\xdcs\x92\xd7\xd9\xb2\x14w-\x08+\x0e\xaawu\xf0Z\x1c\xd4C\x0bNl\x95\x07\xa1\xfe \xf7\xeb!\xc8t{\x9e<b\xaa\xe9\xba\xc1\x98\xba\x7fY4dH4gZ\xcd\x7f\xba\x9b\xf4=\xcc\x1bl\xeb\xbe2\x8a\xfb\x95\xea/\x9e\nM\x10C\r\x95\x1b\xd9\xb7A\x02\x143\x1b\xa4\x13\x04\xd6\x03(RX\x17(\x83\xa9\xf2\xffK\x07\xa5\x85q\x12\xd9\xc9_#VbL\xbe\xf9\x9f\xef\xa1\x85\x1aU\n\xae\n\n\x97\xb0F\x91\x11\x8c\x94\x165\x16\x06\xd9\xef\xd4\xb4\x17v\xe4\xa1\xf2\x95\x9d\xa0\x01\x9e]\n[!+\x10\xa4]\xcf\xf2\x9c\xac=\xfc\x9fW\xac\xdb\xf5lQ\xf9\xb3\x9d\xfe\xc9\xec\x1dk\x90Z\x01\xefJ\x1d\xbcl:2\xf7\xc0s\xda\xf2\xa1>\x00\x1a#-\x19%(\xf5\x1b\x19\xff\xb3\xdb,.\x0b\x08`\x8f\xd8\x81\xe8D\xe2\x96\n\xc6,\x9c\xf2\xcda!\x18[\xf8\x1aO\x9d\x97\x95\xa80\x99xU\x0f\x98\x10]!F\x94\x18\xdclo\xe9l-\x00\\\x98\xee,\x1bw\xe0\x88\x05\xf2\xc0\xa2\x88\xc5\x9b\x8f\x9fv-^(\x90\xbf\xf12\xb3\xcb\xe9\xabl\x8c\x8bN\x90\xc6\xc7\xdd|\xcb\x9e}[\xf9\xff=X\x91b}?\xcd\xfe\\\xec\xb2\x9f\xde\xb4\xaaW\xc9J\x8f/|,\x0

In [27]:
print(pyaes.Counter(iv))
#Et du coup de le regénéré
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext))

b"Le march\xe9 des sneakers en France a connu une croissance exponentielle au cours des derni\xe8res ann\xe9es, devenant un ph\xe9nom\xe8ne de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumi\xe8re ses dynamiques \xe9conomiques, culturelles et sociales.\nD'un point de vue \xe9conomique, le commerce de sneakers en France est stimul\xe9 par une demande croissante pour des mod\xe8les rares et exclusifs. \nLes \xe9ditions limit\xe9es et les collaborations entre marques de renom et designers c\xe9l\xe8bres cr\xe9ent une urgence d'achat, poussant les prix sur le march\xe9 secondaire \xe0 atteindre des sommets vertigineux. \nCette situation a engendr\xe9 l'\xe9mergence d'une v\xe9ritable \xe9conomie parall\xe8le, o\xf9 les sneakers sont non seulement des objets de mode, mais \xe9galement des actifs sp\xe9culatifs.\n\nSur le plan culturel, les s

In [28]:
p = 7946851324679854613245823
g = 5
A = 7579501795988122393422986
## Elle publie A
password = "35426517"
IV = "756431" 
N = int(password + "0000" + IV) 

b = 123456789  

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  # Clé secrète partagée
C = (K * N) % p  # Chiffrement de N avec la clé secrète partagée

B, C

(387225946877039798412155, 5883019579684883513940408)

## Exercice 3

#### 1. Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).

In [29]:
# Fonction pour calculer l'inverse modulo
def inverse_modulo(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Fonction pour calculer la signature
def sign_elgamal(m, x, p, g):
    import random
    # Sélection d'un nombre aléatoire k
    k = random.randint(2, p - 2)
    while pow(k, -1, p - 1) == 0:  # Assure que k est premier avec p-1
        k = random.randint(2, p - 2)
    r = pow(g, k, p)
    k_inv = inverse_modulo(k, p - 1)
    s = (m - x * r) * k_inv % (p - 1)
    return r, s

# Fonction pour vérifier la signature
def verify_elgamal(m, r, s, y, p, g):
    v1 = pow(g, m, p)
    v2 = (pow(y, r, p) * pow(r, s, p)) % p
    return v1 == v2

# Paramètres ElGamal
p = 257
g = 3
x = 97

# Clé publique
y = pow(g, x, p)

# Message à signer
m = 123

# Signature
r, s = sign_elgamal(m, x, p, g)

# Vérification de la signature
valid = verify_elgamal(m, r, s, y, p, g)

print("Signature valide :", valid)


Signature valide : True


#### 2. Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

Supposons que nous ayons un fichier texte appelé "format.txt" contenant le message à signer.



**a) Génération des clés RSA :**
Tout d'abord, nous devons générer une paire de clés RSA privée/publique à l'aide d'OpenSSL.

In [30]:
#copy paste in a terminal
openssl genpkey -algorithm RSA -out private_key.pem
openssl rsa -pubout -in private_key.pem -out public_key.pem

SyntaxError: invalid syntax (1953313993.py, line 2)

Ces commandes génèrent une clé privée private_key.pem et une clé publique correspondante `public_key.pem`.



**b) Signature du message :**

Ensuite, nous signons le message avec notre clé privée.

In [ ]:
#copy paste in a terminal
openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt

SyntaxError: invalid syntax (4259996256.py, line 1)

Cela crée une signature du message dans le fichier signature.bin.


**c) Vérification de la signature :**

Pour vérifier la signature avec la clé publique correspondante, nous utilisons :

In [ ]:
#copy paste in a terminal
openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

SyntaxError: invalid syntax (1276513542.py, line 1)

Si la vérification réussit, OpenSSL affichera "Verified OK".


Cet exemple montre comment utiliser OpenSSL pour générer une paire de clés RSA, signer un message avec la clé privée et vérifier la signature avec la clé publique. Vous pouvez répéter ces étapes dans un terminal pour expérimenter avec vos propres messages et clés.

In [ ]:
(base) Appliedcrypto on  main 1×🗑️ 1×📝 4×📌 via 🐍 v3.8.8 via 💎 v3.0.0 ➜ openssl genpkey -algorithm RSA -out private_key.pem
openssl rsa -pubout -in private_key.pem -out public_key.pem
.....+++++
............................................................................+++++
writing RSA key
(base) Appliedcrypto on  main 1×🗑️ 1×📝 4×📌 via 🐍 v3.8.8 via 💎 v3.0.0 ➜ openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt       
(base) Appliedcrypto on  main 1×🗑️ 1×📝 4×📌 via 🐍 v3.8.8 via 💎 v3.0.0 ➜ openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt
Verified OK

SyntaxError: invalid non-printable character U+E0A0 (4283086991.py, line 1)